# Boosting Models

## Revision on Decision Trees and Random Forest

### Decision Trees

* Decision Tree တွေဟာ human တွေအနေနဲ့ ဖတ်လို့ လွယ်တယ်
* Decision Tree တွေဟာ model assumption မရှိသလောက်မို့ Bias နည်းတယ်
  * Decision Tree တခု performance မကောင်းရင် training data နဲ့ testing data မတူလို့ ဖြစ်ဖို့ အရမ်းများတယ်။
* Decision Tree တွေဟာ branching ခွဲဖို့အတွက် criterion အဖြစ် gini index ကို လည်းကောင်း၊ entropy ကိုသော်လည်းကောင်း သုံးတယ်။
* Decision Tree တွေဟာ training data တွေကို တိတိကျကျ fit နိုင်တာမို့လို့ **strong learner** တွေဖြစ်ကြတယ်။

### Random Forest

* Decision Tree တခုဟာ Bias မရှိသလောက်နည်းပြီး Variance များတတ်တယ်ဆိုတော့ အဲဒီ ပြဿနာကို ဖြေရှင်းဖို့ Random Forest ကို သုံးကြတယ်။
* Variance ရဲ့ ပြဿနာဟာ Training data ကို overfit လို့ (training data နဲ့ exposure များလို့) ဖြစ်တာမို့လို့ 
* Random Forest မှာ feature အကုန် မသုံး၊ sample အကုန် မသုံးတဲ့ Decision Tree တွေ အများကြီးဆောက်ပြီး သူတို့ရဲ့ result ကို voting ဖြစ်စေ၊ average ဖြစ်စေ ယူတာဖြစ်တယ်။
* Random Forest မှာ ပါတဲ့ Tree တခုချင်းစီဟာ training data အကုန်မသုံးပေမဲ့ **strong learner** -- full height tree တွေ ဖြစ်တယ်။
* Random Forest မှာ ပါတဲ့ Tree တခုချင်းစီဟာ size မတူပေမဲ့ voting right တူကြတယ် (တပင် တမဲ)။
* Random Forest မှာ ပါတဲ့ Tree တခုချင်းစီဟာ creation order ကို ဂရုစိုက်စရာ မလိုဘူး။

## Adaptive Boosting

3 Main Ideas:

1. Adaboost combines **weak learners** -- stump (or tree of depth=2, no. of leaves=2) -- to create a **strong learner**
2. Voting is NOT uniform. Instead, it uses **weighted voting** and
3. Order of creating stumps matters.

**Boosting model တွေဟာ **weak learner** တွေကို စုပေါင်းပြီး boost လိုက်လို့ **strong learner** ရလာရတယ်လို့ အစွဲပြုပြီး သူတို့ နာမည်ကို ရလာတာ ဖြစ်တယ်**

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

import numpy as np
from sklearn import ensemble, datasets, metrics, model_selection as model_selection, cluster

In [ ]:
# 1. load the data
df_X, ds_y = datasets.load_digits(n_class=2, return_X_y=True, as_frame=True)

# 2. split into train and test sets
tr_X, ts_X, tr_y, ts_y = model_selection.train_test_split(df_X, ds_y, random_state=42)

# 3. some feature engineering
kmean = cluster.KMeans()
feat_tr_X = kmean.fit_transform(tr_X)
feat_ts_X = kmean.transform(ts_X)

# 4. build adaboost model
adb_model = ensemble.AdaBoostClassifier(n_estimators=50, learning_rate=0.8, random_state=24)
adb_model.fit(feat_tr_X, tr_y)

# 5. test the model
pred_y = adb_model.predict(feat_ts_X)

print (metrics.classification_report(y_true=ts_y, y_pred=pred_y))

Adaptive Boosting ဟာ stump တခုချင်းစီအတွက်

* voting weight ကို $\frac{1}{2} log(\frac{1-E}{E})$ လို့ တွက်တယ်။ 
  * ဒီနေရာမှာ $E$ က total weighted error (weighted odd ratio of no error to error) ဖြစ်တယ်။
  * bad classifier မှာ ၂ မျိုးရှိတယ်။ 
    * ပထမတမျိုးက တဝက်မှန် တဝက်မှား၊ သူ့အတွက်က voting weight $\frac{1}{2} log(\frac{1-E}{E}) = \frac{1}{2} log(\frac{1/2}{1/2}) = \frac{1}{2} log(1) = 0$
    * ဒုတိယတမျိုးက အမြဲ ဇောက်ထိုး ပြောနေသူ၊ သူ့အတွက် voting weight က large negative value ဖြစ်တယ်။ သဘောကတော့ သူ vote တာရဲ့ ပြောင်းပြန်ကို ယူမယ်ဆိုတဲ့ သဘောပဲ။

* total weighted error ကို တွက်ဖို့ sample weight ကို သုံးတယ်။ 
  * stump တခု ဆောက်ပြီးတိုင်း သူမှားတဲ့ sample တွေကို နောက် stump တွေက ပြင်ဖို့ လိုတာမို့ မှားတဲ့ sample weight တွေကို $e^{\text{voting weight}}$ နဲ့ မြှောက်ပစ်တယ်။
    * သဘောက good classifier မှားတဲ့ sample ကို အရေးကြီးအဆင့်အဖြစ် သတ်မှတ်တယ်ဆိုတဲ့သဘောပဲ။
  * သူမှန်တဲ့ sample တွေကိုတော့ $e^{\text{voting weight}}$ နဲ့ စားပစ်တယ်။
    * သဘောက good classifier မှန်ပြီးသား sample ကို နောက်တဆင့်မှာ အမှားပြင်စရာ မလိုတော့ဘူးဆိုတဲ့ သဘောပဲ။
  * အားလုံးပြီးရင် sample weight တွေ အကုန်ပေါင်းရင် 1 ရအောင် normalize ပြန်လုပ်ရတယ်။

* sample weight တွေကို update ပြီးတဲ့ အချိန်မှာ အဲဒီ weight တွေအတိုင်း sample ပြန်ကောက်ပြီး နောက် tree (stump) ကို ဆောက်တယ်။

In [9]:
# try adaboost with more datasets
df_X, ds_y = datasets.load_breast_cancer(return_X_y=True, as_frame=True)

array([0, 1])

In [10]:
# try adaboost with more datasets
df_X, ds_y = datasets.load_wine(return_X_y=True, as_frame=True)

array([0, 1, 2])

## Gradient Boosting

* Gradient Boost ဟာ Adaptive Boosting နဲ့ တော်တော်လေး ဆင်ပါတယ်။
* Gradient Boost က အရွက် တရွက်တည်းနဲ့ စတယ်။ ပြီးတော့ သူ့ tree တွေက stump size ထက် ပိုကြီးပါတယ်။
* Gradient Boost က resampling, sample weight, total weighted error တွေ မတွက်ပဲ အရင် tree ရဲ့ error ကို နောက် tree က predict လုပ်ဖို့ပဲ ကြိုးစားတယ်။

* `sklearn.ensemble` က `GradientBoostingClassifier` ဟာ `XGBoost` နဲ့ `lightgbm` တို့နဲ့ အလုပ်လုပ်ပုံ၊ သင်္ချာသဘောတရား တူတူပါပဲ။
  * `HistGradientBoostingClassifier` ကတော့ `GradientBoostingClassifier` ထက် အများကြီး ပိုမြန်ပါတယ်။

* Learning rate က Gradient Boost ရဲ့ tree တွေရဲ့ voting weight ကို control လုပ်ပေးတယ်။
  * ဖြည်းဖြည်းပဲ learn တာဟာ Variance ကို အမြန်ကြီး မတိုးစေပဲ လိုအပ်တဲ့ sweet spot ကို ရောက်တဲ့အခါ ရပ်ဖို့ အခွင့်အရေး ပေးတယ်။

In [12]:
# 1. load the data
df_X, ds_y = datasets.load_digits(n_class=2, return_X_y=True, as_frame=True)

# 2. split into train and test sets
tr_X, ts_X, tr_y, ts_y = model_selection.train_test_split(df_X, ds_y, random_state=42)

# 3. some feature engineering
kmean = cluster.KMeans()
feat_tr_X = kmean.fit_transform(tr_X)
feat_ts_X = kmean.transform(ts_X)

# 4. build model
clf = ensemble.GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=2, max_leaf_nodes=16, random_state=42)
clf.fit(feat_tr_X, tr_y)

# 5. test the model
pred_y = adb_model.predict(feat_ts_X)

print (metrics.classification_report(y_true=ts_y, y_pred=pred_y))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        46
           1       1.00      1.00      1.00        44

    accuracy                           1.00        90
   macro avg       1.00      1.00      1.00        90
weighted avg       1.00      1.00      1.00        90



In [ ]:
# try gradient boosting with more datasets
df_X, ds_y = datasets.load_breast_cancer(return_X_y=True, as_frame=True)